## Imports

In [1]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [3]:
connection = sqlite3.connect('../data/checking-logs.sqlite.sqlite copy')
connection

## Using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
#### time should have the values: after and before
#### avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward
#### only take into account the users that have observations before and after

In [8]:
query = '''
SELECT time, avg(diff) AS avg_diff
FROM (SELECT uid, CAST((JulianDay(test.first_commit_ts) - JulianDay(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS
Integer) as diff, CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before' ELSE 'after' END AS time
        FROM test left join deadlines on test.labname = deadlines.labs
        WHERE labname <> 'project1')
WHERE uid in (SELECT uid
                FROM (SELECT uid, CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before' ELSE 'after' END AS time
                        FROM test LEFT JOIN deadlines ON test.labname=deadlines.labs
                        WHERE labname <> 'project1')
                        GROUP BY uid
                        HAVING COUNT(DISTINCT time) = 2)
GROUP BY time
'''
test_results = pd.io.sql.read_sql(query, connection)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [9]:
query = '''
SELECT time, avg(diff) AS avg_diff
FROM (SELECT uid, CAST((JulianDay(control.first_commit_ts) - JulianDay(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS
Integer) as diff, CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before' ELSE 'after' END AS time
        FROM control left join deadlines on control.labname = deadlines.labs
        WHERE labname <> 'project1')
WHERE uid in (SELECT uid
                FROM (SELECT uid, CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before' ELSE 'after' END AS time
                        FROM control LEFT JOIN deadlines ON control.labname=deadlines.labs
                        WHERE labname <> 'project1')
                        GROUP BY uid
                        HAVING COUNT(DISTINCT time) = 2)
GROUP BY time
'''
control_results = pd.io.sql.read_sql(query, connection)
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


## Close the connection

In [10]:
connection.close()

## have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

#### Yes, it did. If the delta before the first Newsfeed visit is significantly different compared to the delta afterward in the test group, and we do not see the same effect in the control group, then creating the page was a great idea. We can roll it out to the whole group.